# Data Flow Introduction

Data flow is a **static analysis** that looks over a program as a control flow graph, checking how a property (data) goes (flows) through a program. The technique is applied to optimize and reason a program, example:

- Constant-propagation
- Loop invariant
- Common subexpression reduction
- Number of registers needed

This notebook will first present the background needed (graphs and CFG/ICFG) and will explain two examples, the first for Liveness and the second for Reachability.

## Graph

In a nutshell, a graph is a data structure capable of modelling problems by conveerting it to entities (vertexes) and relations (edges). For example, a map can contain cities as vertexes and roads (edges) conecting them.

In [7]:
from graph import *

print(f'[Boa Vista -> Manaus] is a path: {has_path(city_graph(),"Boa Vista", "Manaus")}')
print(f'[Boa Vista -> Manaus] length: {shortest_path_length(city_graph(),"Boa Vista", "Manaus")}')



[Boa Vista -> Manaus] is a path: True
[Boa Vista -> Manaus] length: 740


## Control Flow Graph

It is a way to represent the flow of a program. Depending of the analysis type it can be used as a Basic Block, Functions, Modules and even at instruction level:

```c
int foo(int c) {             // F: foo, Module: test.c
        int a = 0;           // B1        // 1
        int b = 0;                        // 2
    L1: 
        b = a + 1;                        // 3
        c = c + b;           // B2        // 4
        a = b * 2;                        // 5
        if (a < 9)                        // 6

            goto L1;         // B3

        return c;            // B4        // 7
}
```
Here are the CFGs for the Basic Block and Instruction Level 

<table>
<tr>
  <th>Instruction</th>
  <th>Basic Block</th>
</tr>
<tr>
  <th><img src="instr_cfg.png" /></th>
  <th><img src="bb_cfg.png"/></th>
</tr>
</table>


So, what if we want to know how many registers are needed for this program?

- One for each variable?
- Liveness Analysis for variables

## Liveness Analysis

A variable is considered **live** if its values is going to be used in the future, otherwise its considered **dead**. If two variables are not alive at the same time then they can share a register. For example, the program below only needs one register 

```
int a = 0;            // r0 = 0
int b = a + 2;        // r0 = r0 + 2;
int c = b + 1;        // r0 = r0 + 1
return c;             // r0
```

### More background...

First we need to define some functions over a Graph and CFG:

- $G$: Graphs<br>
  $V$: Vertexes<br>
  $L$: Labels/Strings

- **pred** : $G \rightarrow V \rightarrow 2^V$ <br>
  **pred(G, v)**: Returns every vertex that immediatly preceddes **v** 

- **succ** : $V \rightarrow 2^V$ <br>
  **succ(G, v)**: Returns every vertex that immediatly succedes **v** 
  
- **gen** : $G \rightarrow V \rightarrow 2^L$ <br>
  **gen(G, v)** : Returns every variable that is **generated** at vertex v. In liveness this mean when a variable is USED


- **kill** : $G \rightarrow V \rightarrow 2^L$ <br>
  **kill(G, v)** : Returns every variables that is **killed** at vertex v. Is liveness this means when a variable is DEFINED



In [8]:
A = CFG()

# Add node expects (id, [kill], [gen)
A.add_node(1, ["a"], [])            # 1. a = 0
A.add_node(2, ["b"], [])            # 2. b = 0
A.add_node(3, ["b"], ["a"])         # 3. b = a + 1
A.add_node(4, ["c"], ["c", "b"])    # 4. c = c + b
A.add_node(5, ["a"], ["b"])         # 5. a = b * 2
A.add_node(6, [], ["a"])            # 6. a < 9
A.add_node(7, [], ["c"])            # 7. return c

A.add_edges(
    [(1,2), (2,3), (3,4), (4,5), (5,6), (6,7), (6,3)] # Same as above...
)

A.verbose = True
# Examples
A.pred(3)
A.succ(6)
A.kill(2)
A.gen(4)

A.verbose = False


Predecessors of 3 are: 2, 6
Successors of 6 are: 7, 3
Vertex '2'' kills: b
Vertex '4'' gen: c, b


## Liveness of a variable:

### Edges
A variable X is considered alive if there exists a *directed path* in the graph if there exists a 
path from that edge to a use of X and that path does not goes through any definition


### Nodes
- **live-out**: if it is alive on any outter edges
- **live-in**: if it is alive on any in edges

### Computing Liveness (classic)

1. If a variable is in (cfg, v), it is live-in at the vertex v
2. If a variable is live-in at a vertex v, the it is live-out in all vertexes pred(cfg, v)
3. If a variable is live-out at a vertex and not in def_vertex(cfg, v), then the variable is also live-in at v

Dataflow-equations: 
- live_in(v) = gen(cfg,v) $\cup$ (out_vertex(v) - kill(v)) 
- live-out(v) = $\cup_{s \in succ(cfg,v)}$ live-in(s)

**Note:** liveness flows backwards.



In [9]:
import pandas as pd # For printing the matrix

def compute_backwards(cfg: CFG, max_iter = 15, reverse=False):
    live_in = dict()
    live_out = dict()
    
    for v in cfg.G.nodes: # INITIALIZE SOLUTIONS
        live_in[v] = set() # empty set
        live_out[v] = set() # empty set

    iteraction = 1
    converged = False # THE SOLUTION WAS NOT FOUND YET    
    while not converged:
        converged = True # hope for the best...
        vertex_iterator = list(cfg.G.nodes) if not reverse else reversed(list(cfg.G.nodes))

        for v in vertex_iterator: # for all vertexes v in G
            old_live_in = set(live_in[v])     # Copy
            old_live_out = set(live_out[v])   # Copy

            # Compute live_in[v]
            live_in[v] = set(cfg.gen(v))
            for x in live_out[v].difference(cfg.kill(v)):
                live_in[v].add(x)
            
            # Compute live_out[v]
            for s in cfg.succ(v):
                for x in live_in[s]:
                    live_out[v].add(x)

            # Did it converge?
            not_in_old_in = live_in[v].difference(old_live_in)
            not_in_old_out = live_out[v].difference(old_live_out)        
            if (len(not_in_old_in) > 0) or (len(not_in_old_out) > 0):
                converged = False # Solution was not found yet...
        # What if this never coverges?
        if iteraction > max_iter:
            print("Giving up...")
            return

        # Try one more time...
        iteraction = iteraction + 1
    print(f"Found solution at iteration: {iteraction-1}")
    print(f"Live-in: {live_in}")
    print(f"Live-out {live_out}")        
    return live_in, live_out

compute_backwards(A, 10, reverse=False)

Found solution at iteration: 8
Live-in: {1: {'c'}, 2: {'a', 'c'}, 3: {'a', 'c'}, 4: {'b', 'c'}, 5: {'b', 'c'}, 6: {'a', 'c'}, 7: {'c'}}
Live-out {1: {'a', 'c'}, 2: {'a', 'c'}, 3: {'b', 'c'}, 4: {'b', 'c'}, 5: {'a', 'c'}, 6: {'a', 'c'}, 7: set()}


({1: {'c'},
  2: {'a', 'c'},
  3: {'a', 'c'},
  4: {'b', 'c'},
  5: {'b', 'c'},
  6: {'a', 'c'},
  7: {'c'}},
 {1: {'a', 'c'},
  2: {'a', 'c'},
  3: {'b', 'c'},
  4: {'b', 'c'},
  5: {'a', 'c'},
  6: {'a', 'c'},
  7: set()})